In [3]:
import gffutils

# 1. 先连接数据库文件，生成对象
db = gffutils.FeatureDB("/data/work/hg38_annotations.db")

In [4]:
import os
import pickle
import pysam
import pandas as pd
import gffutils

def get_gene_structure_dict(db, cache_path="gene_structure_cache.pkl"):
    """
    如果缓存文件存在则加载，否则构建并保存。
    """
    if os.path.exists(cache_path):
        print(f"--- 发现缓存文件，正在从 {cache_path} 加载基因结构 ---")
        with open(cache_path, 'rb') as f:
            return pickle.load(f)

    print("--- 未发现缓存，开始构建基因结构字典 (仅需运行一次) ---")
    gene_structures = {}
    
    # 获取总基因数用于进度展示（可选）
    total_genes = db.count_features_of_type('gene')
    count = 0

    for gene in db.features_of_type('gene'):
        g_id = gene.id  
        
        last_exons = set()
        all_exons = set()
        
        # 遍历该基因下的所有转录本
        for tx in db.children(gene, featuretype='transcript'):
            # 获取该转录本的所有外显子，并按坐标排序
            exons = sorted(list(db.children(tx, featuretype='exon')), key=lambda e: e.start)
            if not exons: 
                continue
                
            # 根据基因链方向确定 3' 末端外显子
            # 正链：最后一个外显子；负链：第一个外显子
            target_last = exons[-1] if gene.strand == '+' else exons[0]
            last_exons.add((target_last.start, target_last.end))
            
            for e in exons:
                all_exons.add((e.start, e.end))
        
        gene_structures[g_id] = {
            'strand': gene.strand,
            'last_exons': list(last_exons),
            'all_exons': list(all_exons)
        }
        
        count += 1
        if count % 5000 == 0:
            print(f"已处理 {count}/{total_genes} 个基因...")

    # 保存到磁盘
    print(f"--- 正在保存缓存到 {cache_path} ---")
    with open(cache_path, 'wb') as f:
        pickle.dump(gene_structures, f)
        
    return gene_structures

In [5]:
cache_file = get_gene_structure_dict(db, cache_path="gene_structure_cache.pkl")

--- 未发现缓存，开始构建基因结构字典 (仅需运行一次) ---
已处理 5000/36601 个基因...
已处理 10000/36601 个基因...
已处理 15000/36601 个基因...
已处理 20000/36601 个基因...
已处理 25000/36601 个基因...
已处理 30000/36601 个基因...
已处理 35000/36601 个基因...
--- 正在保存缓存到 gene_structure_cache.pkl ---
